### Decision Tree Exercise
- Following example of [this blog post](https://towardsdatascience.com/decision-tree-from-scratch-in-python-46e99dfea775)
- [Simplified version](https://github.com/joachimvalente/decision-tree-cart/blob/master/minimal_cart.py#L14-L79)

In [1]:
# define Node class, which will be used in the classifier
class Node:
    def __init__(self, gini, n_samples, n_samples_class, pred_class):
        self.gini = gini
        self.n_samples = n_samples
        self.n_samples_class = n_samples_class
        self.pred_class = pred_class
        self.col_idx = 0
        self.val = 0
        self.left = None
        self.right = None

In [2]:
# decision tree on categorical target values
class DecisionTreeClassifier():
    # constructor function. define the maximum depth of the tree
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        
    # fit function. bind input data matrix & target value
    def fit(self, X, y):
        # set meta data of input matrix & target classes
        self.n_class = len(set(y))
        self.n_feature = X.shape[1]
        # recursively call grow_tree function to split input data. returns a node class
        self.tree = self.grow_tree(X, y)
    
    # function that finds the best split point. return the column index & value in that column
    def best_split(self, X, y):
        m = len(y)
        if m <= 1:
            return None, None
        # count values by class
        num_parent = [np.sum(y==c) for c in range(self.n_class)]
        # calculate the parent gini impurity
        best_gini = 1. - np.sum([(n / m)**2 for n in num_parent]) 
        # initialize best column and best value
        best_col, best_val = None, None
        # loop through each column
        for col in range(self.n_feature):
            #  sort the current column in asc
            idx = np.argsort(X[:, col])
            vals = X[:, col][idx]
            labels = y[idx]
            # to begin with, put all values on the right side, then move value one by one to the left sid
            # initialize count by class on the left to be all zeros
            left_cnt = [0] * self.n_class
            right_cnt = num_parent.copy()
            # loop through each possible split point
            # skipping the 1st because all values are on the right to begin with
            for  i in range(1, m):
                label = labels[i-1]
                left_cnt[label] += 1
                right_cnt[label] -= 1
                # calculate gini impurity on both sides
                left_gini = 1. - np.sum([(left_cnt[x] / i)**2 for x in range(self.n_class)])
                right_gini = 1. - np.sum([(right_cnt[x] /(m- i))**2 for x in range(self.n_class)])
                # calculate total gini, which is weighted sum of both sides
                gini = (i/m) * left_gini + ((m-i)/m) * right_gini
                # skip if 2 values are the same
                if vals[i] == vals[i-1]:
                    continue
                # if the current gini is smaller than the parent gini
                # update the best column & best value
                if gini < best_gini:
                    best_gini = gini
                    best_col = col
                    best_val = (vals[i] + vals[i-1])/2
        return best_col, best_val
    
    def gini_impurity(self, y):
        m = len(y)
        return 1. - np.sum([(np.sum(y==c)/m)**2 for c in range(self.n_class)])
    
    def grow_tree(self, X, y, depth = 0):
        # count samples by class
        n_samples_class = [np.sum(y==i) for i in range(self.n_class)]
        # set predicted class to be the one with the largest sample counts
        pred_class = np.argmax(n_samples_class)
        # create a new node with summary of the input set
        node = Node(
            gini = self.gini_impurity(y) # calculate parent gini 
            ,n_samples = len(y) # record total sample size
            ,n_samples_class = n_samples_class # sample size by class
            ,pred_class = pred_class # set predicted class
        )
        # check if the max depth has been reached
        if depth < self.max_depth:
            # if not reached, call best_split to run through all columns and find the best column & value to split
            col, val = self.best_split(X, y)
            if col is not None:
                # split the whole dataset in this node into left & right based on the best column & value
                left_idx = X[:, col] < val
                X_left, y_left = X[left_idx], y[left_idx]
                X_right, y_right = X[~left_idx], y[~left_idx]
                # set the column index & value attributes of the current node
                node.col_idx = col
                node.val = val
                # call grow_tree function recursively to further split dataset and create nodes
                node.left = self.grow_tree(X_left, y_left, depth+1)
                node.right = self.grow_tree(X_right, y_right, depth+1)
        return node 
    
    # function that makes prediciton for each sample
    def predict_row(self, row):
        node = self.tree
        # check if at leaf (leaf node has no further split)
        while node.left:
            # compare input sample with the best value on the vest column
            if row[node.col_idx] < node.val:
                # if smaller, go down the tree along the left
                node = node. left
            else:
                node = node.right
        return node.pred_class
          
    def predict(self, X):
        # call predict_row to make prediction for each sample
        return [self.predict_row(row) for row in X]

In [3]:
# load data for testing

from sklearn.datasets import load_iris
from sklearn.metrics import confusion_matrix

data = load_iris()
X, y = data.data, data.target

In [4]:
clf = DecisionTreeClassifier(max_depth=4)
clf.fit(X, y)
y_pred = clf.predict(X)
confusion_matrix(y, y_pred)

array([[50,  0,  0],
       [ 0, 50,  0],
       [ 0,  1, 49]])